In [3]:
from src.utils import gallery_specs, gallery_notations
from src.tokens import load_tokens
import random
from src.distances import dmat_and_order
import numpy as np
import plotly.express as px

random.seed(42)

In [4]:
gallery = "movies"
distance = "cd"

In [5]:
all_specs = gallery_specs(gallery)
tokens = load_tokens()

In [7]:
notations = []
remoteness = []
vocab = []
for i in range(1000):
  sample = random.choices(all_specs, k=len(all_specs))
  for notation in gallery_notations(gallery):
    notations.append(notation)
    dmat_sym, order = dmat_and_order(gallery, notation, distance)
    sample_idx = np.array([order.index(s) for s in sample])
    remoteness.append(np.mean(np.median(dmat_sym[sample_idx][:,sample_idx], axis=1)))
    vocab.append(tokens.query("gallery == @gallery and notation == @notation and spec in @sample")["token"].nunique())

In [8]:
px.density_contour(x=vocab, y=remoteness, color=notations, width=800, height=800, labels=dict(y='mean remoteness', x='vocabulary size'))